## Python kernal

## Trim

In [ ]:
import glob
for i in glob.glob("reads/leaf_tissue/raw/*_1.fq.gz"):
    o = i.split("_1.fq.gz")[0]+"_2.fq.gz"
    !trim_galore -o reads/leaf_tissue/processed/ -j 8 --paired {i} {o}

## Align

In [ ]:
!kallisto index -i ref/GCF_007990345.1_Gossypium_hirsutum_v2.1_rna_cotton.idx ref/GCF_007990345.1_Gossypium_hirsutum_v2.1_rna_cotton.fna

In [ ]:
import glob
cotton = {"R1","R2","R3","R4","R5","R6","R7","R8","R9","R10","R11","R12"}
for i in glob.glob("reads/leaf_tissue/processed/*_1.fq.gz"):
    f = i.split("/")[-1].split("-")[0]
    o = i.split("1_val_1.fq.gz")[0]+"2_val_2.fq.gz"
    if f in cotton:
        print("cotton", f)
        !kallisto quant -t 16 -i ref/GCF_007990345.1_Gossypium_hirsutum_v2.1_rna_cotton.idx -b 100 -o aligned/cotton/{f} {i} {o}

## R kernal

In [ ]:
library("tximport")
library("readr")
library("tximportData")
library("DESeq2")
library("pheatmap")
library("ggplot2")

## Expression analysis

In [ ]:
sample_id = dir(file.path("aligned", "cotton"))
kal_dirs = file.path("aligned", "cotton", sample_id)
files = file.path("aligned/cotton", sample_id, "abundance.h5")
names(files) = sample_id
txi.kallisto <- tximport(files, type = "kallisto", txOut = TRUE)
samples = read.table("aligned/cotton_one_factor.txt", header=TRUE)
rownames(samples) = samples$Sample
ddsTxi = DESeqDataSetFromTximport(txi.kallisto,
                                   colData = samples,
                                   design = ~treatment)
keep = rowSums(counts(ddsTxi)) >= 10
dds = ddsTxi[keep,]
dds$treatment <- relevel(dds$treatment, ref = "water")
dds = DESeq(dds)

### BioClay vs. water

In [ ]:
res05 = results(dds, contrast=c("treatment","BioClay","water"), alpha=0.05)
resOrdered = res05[order(res05$pvalue),]
resSig = subset(resOrdered, padj < 0.05)
write.csv(as.data.frame(resSig), 
          file="treatment_BioClay_vs_water.csv")
rld = rlog(dds, blind=FALSE)
options(repr.plot.width=7, repr.plot.height=7)
select_genes<-rownames(resOrdered[1:50,])
pheatmap(assay(rld)[ select_genes,],
         cluster_rows=T, 
         fontsize = 15, 
         filename = "bioclay_heatmap.png", 
         width=10, 
         height = 10.5)

### LDH vs. water

In [ ]:
res05 = results(dds, contrast=c("treatment","LDH","water"), alpha=0.05)
resOrdered = res05[order(res05$pvalue),]
resSig = subset(resOrdered, padj < 0.05)
write.csv(as.data.frame(resSig), 
          file="treatment_LDH_vs_water.csv")
rld = rlog(dds, blind=FALSE)
options(repr.plot.width=7, repr.plot.height=7)
select_genes<-rownames(resOrdered[1:50,])
pheatmap(assay(rld)[ select_genes,],
         cluster_rows=T, 
         fontsize = 15, 
         filename = "ldh_heatmap.png", 
         width=10, 
         height = 10.5)

### Naked vs. water

In [ ]:
res05 = results(dds, contrast=c("treatment","naked_dsRNA","water"), alpha=0.05)
resOrdered = res05[order(res05$pvalue),]
resSig = subset(resOrdered, padj < 0.05)
write.csv(as.data.frame(resSig), 
          file="treatment_naked_dsRNA_vs_water.csv")
rld = rlog(dds, blind=FALSE)
options(repr.plot.width=7, repr.plot.height=7)
select_genes<-rownames(resOrdered[1:50,])
pheatmap(assay(rld)[ select_genes,],
         cluster_rows=T, 
         fontsize = 15, 
         filename = "naked_heatmap.png", 
         width=10, 
         height = 10.5)

## PCA plot

In [ ]:
vsd = vst(dds, blind=FALSE)
pcaData = plotPCA(vsd, intgroup=c("treatment"), returnData=TRUE)
percentVar = round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=treatment)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed() + theme(text = element_text(size = 17)) 
ggsave("pca.png", width = 7, height = 7)

## Python kernal

## Counts - upreg

In [ ]:
import csv
first = True
naked_upreg = set()
naked_downreg = set()
with open('treatment_naked_dsRNA_vs_water.csv', newline='') as csvfile:
    r = csv.reader(csvfile)
    for row in r:
        if first:
            first=False
        else:
            if float(row[2]) > 1:
                naked_upreg.add(row[0])
            elif float(row[2]) < 1:
                naked_downreg.add(row[0])
first = True
bioclay_upreg = set()
bioclay_downreg = set()
with open('treatment_BioClay_vs_water.csv', newline='') as csvfile:
    r = csv.reader(csvfile)
    for row in r:
        if first:
            first=False
        else:
            if float(row[2]) > 1:
                bioclay_upreg.add(row[0])
            elif float(row[2]) < 1:
                bioclay_downreg.add(row[0])
first = True
ldh_upreg = set()
ldh_downreg = set()
with open('treatment_LDH_vs_water.csv', newline='') as csvfile:
    r = csv.reader(csvfile)
    for row in r:
        if first:
            first=False
        else:
            if float(row[2]) > 1:
                ldh_upreg.add(row[0])
            elif float(row[2]) < 1:
                ldh_downreg.add(row[0])

In [ ]:
bc_ldh_upreg = bioclay_upreg.intersection(ldh_upreg)
all_upreg = bc_ldh_upreg.intersection(naked_upreg)
bc_ldh_only = bc_ldh_upreg - all_upreg
bc_naked_upreg = bioclay_upreg.intersection(naked_upreg)
bc_naked_only = bc_naked_upreg - all_upreg
ldh_naked_upreg = ldh_upreg.intersection(naked_upreg)
ldh_naked_only = ldh_naked_upreg - all_upreg
ldh_only = ldh_upreg - all_upreg - bc_ldh_only - ldh_naked_only
bc_only = bioclay_upreg - all_upreg - bc_ldh_only - bc_naked_only
naked_only = naked_upreg - all_upreg - bc_naked_only - ldh_naked_only

## Check function of LDH intersect BioClay upreg transcripts

In [ ]:
a = RefSeq()
a.load_ref_file("ref/GCF_007990345.1_Gossypium_hirsutum_v2.1_rna_cotton.fna")
for i in a.keys():
    if i.split()[0] in bc_ldh_only:
        print("{0} - {1}".format(i.split()[0], i.split("PREDICTED: Gossypium hirsutum ")[1]))

## Counts - downreg

In [ ]:
first = True
naked_upreg = set()
naked_downreg = set()
with open('treatment_naked_dsRNA_vs_water.csv', newline='') as csvfile:
    r = csv.reader(csvfile)
    for row in r:
        if first:
            first=False
        else:
            if float(row[2]) > 1:
                naked_upreg.add(row[0])
            elif float(row[2]) < 1:
                naked_downreg.add(row[0])
first = True
bioclay_upreg = set()
bioclay_downreg = set()
with open('treatment_BioClay_vs_water.csv', newline='') as csvfile:
    r = csv.reader(csvfile)
    for row in r:
        if first:
            first=False
        else:
            if float(row[2]) > 1:
                bioclay_upreg.add(row[0])
            elif float(row[2]) < 1:
                bioclay_downreg.add(row[0])
first = True
ldh_upreg = set()
ldh_downreg = set()
with open('treatment_LDH_vs_water.csv', newline='') as csvfile:
    r = csv.reader(csvfile)
    for row in r:
        if first:
            first=False
        else:
            if float(row[2]) > 1:
                ldh_upreg.add(row[0])
            elif float(row[2]) < 1:
                ldh_downreg.add(row[0])

In [ ]:
bc_ldh_downreg = bioclay_downreg.intersection(ldh_downreg)
all_downreg = bc_ldh_downreg.intersection(naked_downreg)
bc_ldh_only = bc_ldh_downreg - all_downreg
bc_naked_downreg = bioclay_downreg.intersection(naked_downreg)
bc_naked_only = bc_naked_downreg - all_downreg
ldh_naked_downreg = ldh_downreg.intersection(naked_downreg)
ldh_naked_only = ldh_naked_downreg - all_downreg
all_downreg = ldh_naked_downreg.intersection(bioclay_downreg)
ldh_only = ldh_downreg - all_downreg - bc_ldh_only - ldh_naked_only
bc_only = bioclay_downreg - all_downreg - bc_ldh_only - bc_naked_only
naked_only = naked_downreg - all_downreg - bc_naked_only - ldh_naked_only

In [ ]:
for i in a.keys():
    if i.split()[0] in bc_ldh_only:
        print("{0} - {1}".format(i.split()[0], i.split("PREDICTED: Gossypium hirsutum ")[1]))

## R kernal

## Euler plots

In [ ]:
library(eulerr)
fit1 = euler(c("LDH" = 40, 
               "Naked" = 28, 
               "BioClay" = 36,
               "LDH&Naked" = 23,
               "LDH&BioClay" = 24,
               "Naked&BioClay" = 29,
               "LDH&Naked&BioClay" = 102
              ),shape="ellipse")
png(file="upregreg_eular.png")
plot(fit1,
     quantities = list(font = 4, fontsize=15),
     lty = 1:3,
     labels = list(font = 4, fontsize=15))
dev.off()

In [ ]:
library(eulerr)
fit1 = euler(c("LDH" = 121, 
               "Naked" = 154, 
               "BioClay" = 207,
               "LDH&Naked" = 9,
               "LDH&BioClay" = 7,
               "Naked&BioClay" = 17,
               "LDH&Naked&BioClay" = 1
              ), shape="ellipse")
png(file="downreg_eular.png")
plot(fit1,
     quantities = list(font = 4, fontsize=15),
     lty = 1:3,
     labels = list(font = 4, fontsize=15),
    )
dev.off()

## Python kernal

## Revigo plot

In [ ]:
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(rc={'figure.figsize':(6,6)}, style="white")
sns.scatterplot(data=df, x="plot_X", y="plot_Y", size="Mortality", hue="Group", alpha=0.8, sizes=(40,800), palette="bright", linewidth = 1)
lgd=plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., frameon=False)
plt.savefig('revego.pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')